In [77]:
# Checking whether the symbols exist in the API
import os
import requests
api_key = os.getenv("ALPHAVANTAGE_API_KEY")
df=pd.read_csv("company_tick_symbols.csv")
for symbol in df['Symbol']:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    if 'Error Message' in data.keys():
        df['Data_Exising']= False
        break
    else:    
        df['Data_Exising']= True
        break
df.to_csv('company_tick_symbols.csv',index=False)

        




In [89]:
# Preprocess data when using Alpha Vantage API and convert it into a DataFrame
def pre_process_data(company_name):
    
    try:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={company_name}&outputsize=full&apikey={api_key}"
        r = requests.get(url)
        data = r.json()
        df=pd.DataFrame(data)
        df1=pd.DataFrame()
        df.reset_index(inplace=True)
        compnay_name=df['Meta Data'][1]
        df.drop(['Meta Data'],axis=1,inplace=True)
        df.drop(index=df.index[0:5],inplace=True)   
        df.reset_index(inplace=True,drop=True)
        df1['Date']=df['index'].apply(lambda x: x)
        df1['Company_Name']=compnay_name
        df1['Open']=df['Time Series (Daily)'].apply(lambda x: x['1. open'])
        df1['High']=df['Time Series (Daily)'].apply(lambda x: x['2. high']) 
        df1['Low']=df['Time Series (Daily)'].apply(lambda x: x['3. low'])
        df1['Close']=df['Time Series (Daily)'].apply(lambda x: x['4. close'])
        df1['Volume']=df['Time Series (Daily)'].apply(lambda x: x['5. volume'])
        return df1
    except ValueError:
        # print("Error in fetching data, check the company name along with the API Key")
        return None
df2=pre_process_data('IBM')


#Add metadata to the company_tick_symbols_processed.csv file: Here the original file is company_symbols.csv

df=pd.read_csv("company_tick_symbols_processed.csv")
df1=pd.read_csv("company_symbols.csv")


for symbol in df['Symbol']:
    df.loc[df['Symbol'] == symbol, 'IPO_Year']=df1['IPO Year'][df1['Symbol']==symbol].values[0]
    df.loc[df['Symbol'] == symbol, 'Country']=df1['Country'][df1['Symbol']==symbol].values[0]
    df.loc[df['Symbol'] == symbol, 'Industry']=df1['Industry'][df1['Symbol']==symbol].values[0]
    
df.to_csv('company_tick_symbols_processed.csv',index=False)

In [46]:
import yfinance as yf
import pandas as pd
import os
import json

# Load the CSV file with company tickers and metadata
df_symbols = pd.read_csv("company_tick_symbols_processed.csv")  # Assume this file has columns like ['Symbol', 'IPO Year', 'Sector', 'Country']

# Create a local directory to store the data files
output_dir = './stock_data/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def save_metadata(company_symbol, metadata):
    # Create a metadata file for the company
    metadata_dir = os.path.join(output_dir, f'company={company_symbol}/')
    os.makedirs(metadata_dir, exist_ok=True)
    
    # Save metadata as a JSON file in the company folder
    metadata_file = os.path.join(metadata_dir, 'metadata.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f)
    # print(f"Metadata for {company_symbol} saved at {metadata_file}")

def save_yearly_data(data, company_symbol, year):
    # Create partitioned directory for company and year
    partition_dir = os.path.join(output_dir, f'company={company_symbol}/year={year}/')
    os.makedirs(partition_dir, exist_ok=True)
    
    # Save the entire year's data to a single Parquet file
    file_path = os.path.join(partition_dir, f'{company_symbol}_data_{year}.parquet')
    data.to_parquet(file_path, compression='snappy')
    print(f"Data for {company_symbol} for year {year} saved at {file_path}")

def fetch_and_store_data_locally(company_symbol, ipo_year, sector, country,company_name,industry):
    # Fetch historical data from 2000-01-01 to 2024-10-01
    stock_data = yf.download(company_symbol, start="2000-01-01", end="2024-10-01")
    
    # Add company metadata and save it
    metadata = {
        "Company Name": company_name,
        "IPO Year": ipo_year,
        "Sector": sector,
        "Country": country,
        "Industry":industry 
    }
    save_metadata(company_symbol, metadata)

    # Group data by year and save yearly data in Parquet format
    stock_data['Year'] = stock_data.index.year
    for year, data in stock_data.groupby('Year'):
        save_yearly_data(data, company_symbol, year)

# Loop through the companies and fetch data
for index, row in df_symbols.iloc[:5].iterrows():
    company_symbol = row['Symbol']
    ipo_year = row['IPO_Year']
    sector = row['Sector']
    country = row['Country']
    company_name=row['Company_Name']
    industry=row['Industry']

    
    print(f"Fetching and storing data for {company_symbol}")
    fetch_and_store_data_locally(company_symbol, ipo_year, sector, country,company_name,industry)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching and storing data for A
Data for A for year 2000 saved at ./stock_data/company=A/year=2000/A_data_2000.parquet
Data for A for year 2001 saved at ./stock_data/company=A/year=2001/A_data_2001.parquet
Data for A for year 2002 saved at ./stock_data/company=A/year=2002/A_data_2002.parquet
Data for A for year 2003 saved at ./stock_data/company=A/year=2003/A_data_2003.parquet
Data for A for year 2004 saved at ./stock_data/company=A/year=2004/A_data_2004.parquet
Data for A for year 2005 saved at ./stock_data/company=A/year=2005/A_data_2005.parquet
Data for A for year 2006 saved at ./stock_data/company=A/year=2006/A_data_2006.parquet
Data for A for year 2007 saved at ./stock_data/company=A/year=2007/A_data_2007.parquet
Data for A for year 2008 saved at ./stock_data/company=A/year=2008/A_data_2008.parquet
Data for A for year 2009 saved at ./stock_data/company=A/year=2009/A_data_2009.parquet
Data for A for year 2010 saved at ./stock_data/company=A/year=2010/A_data_2010.parquet
Data for A 


[*********************100%***********************]  1 of 1 completed


Data for AA for year 2000 saved at ./stock_data/company=AA/year=2000/AA_data_2000.parquet
Data for AA for year 2001 saved at ./stock_data/company=AA/year=2001/AA_data_2001.parquet
Data for AA for year 2002 saved at ./stock_data/company=AA/year=2002/AA_data_2002.parquet
Data for AA for year 2003 saved at ./stock_data/company=AA/year=2003/AA_data_2003.parquet
Data for AA for year 2004 saved at ./stock_data/company=AA/year=2004/AA_data_2004.parquet
Data for AA for year 2005 saved at ./stock_data/company=AA/year=2005/AA_data_2005.parquet
Data for AA for year 2006 saved at ./stock_data/company=AA/year=2006/AA_data_2006.parquet
Data for AA for year 2007 saved at ./stock_data/company=AA/year=2007/AA_data_2007.parquet
Data for AA for year 2008 saved at ./stock_data/company=AA/year=2008/AA_data_2008.parquet
Data for AA for year 2009 saved at ./stock_data/company=AA/year=2009/AA_data_2009.parquet
Data for AA for year 2010 saved at ./stock_data/company=AA/year=2010/AA_data_2010.parquet
Data for A

[*********************100%***********************]  1 of 1 completed


Data for AAON for year 2000 saved at ./stock_data/company=AAON/year=2000/AAON_data_2000.parquet
Data for AAON for year 2001 saved at ./stock_data/company=AAON/year=2001/AAON_data_2001.parquet
Data for AAON for year 2002 saved at ./stock_data/company=AAON/year=2002/AAON_data_2002.parquet
Data for AAON for year 2003 saved at ./stock_data/company=AAON/year=2003/AAON_data_2003.parquet
Data for AAON for year 2004 saved at ./stock_data/company=AAON/year=2004/AAON_data_2004.parquet
Data for AAON for year 2005 saved at ./stock_data/company=AAON/year=2005/AAON_data_2005.parquet
Data for AAON for year 2006 saved at ./stock_data/company=AAON/year=2006/AAON_data_2006.parquet
Data for AAON for year 2007 saved at ./stock_data/company=AAON/year=2007/AAON_data_2007.parquet
Data for AAON for year 2008 saved at ./stock_data/company=AAON/year=2008/AAON_data_2008.parquet
Data for AAON for year 2009 saved at ./stock_data/company=AAON/year=2009/AAON_data_2009.parquet
Data for AAON for year 2010 saved at ./s

[*********************100%***********************]  1 of 1 completed

Data for AAPL for year 2000 saved at ./stock_data/company=AAPL/year=2000/AAPL_data_2000.parquet
Data for AAPL for year 2001 saved at ./stock_data/company=AAPL/year=2001/AAPL_data_2001.parquet
Data for AAPL for year 2002 saved at ./stock_data/company=AAPL/year=2002/AAPL_data_2002.parquet
Data for AAPL for year 2003 saved at ./stock_data/company=AAPL/year=2003/AAPL_data_2003.parquet
Data for AAPL for year 2004 saved at ./stock_data/company=AAPL/year=2004/AAPL_data_2004.parquet
Data for AAPL for year 2005 saved at ./stock_data/company=AAPL/year=2005/AAPL_data_2005.parquet
Data for AAPL for year 2006 saved at ./stock_data/company=AAPL/year=2006/AAPL_data_2006.parquet
Data for AAPL for year 2007 saved at ./stock_data/company=AAPL/year=2007/AAPL_data_2007.parquet
Data for AAPL for year 2008 saved at ./stock_data/company=AAPL/year=2008/AAPL_data_2008.parquet
Data for AAPL for year 2009 saved at ./stock_data/company=AAPL/year=2009/AAPL_data_2009.parquet
Data for AAPL for year 2010 saved at ./s

In [48]:
import pandas as pd
df=pd.read_csv("company_tick_symbols_processed.csv")
len(df)

1062